In [10]:
import pandas as pd
import pandas as pd

brain = pd.read_csv("./data/brain.csv")
main = pd.read_csv("./data/data.csv")


In [29]:
import pickle

brain = pd.read_csv("./data/brain.csv")
brain = brain.drop(brain.columns[[0, 1, 2, 3]], 1)
with open('./data/columns/brain_subcor', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(list(brain), filehandle)

with open('./data/columns/brain_subcor', 'rb') as filehandle:
    # read the data as binary data stream
    brainCol = pickle.load(filehandle)

assert len(brain[brainCol].columns) < len(pd.read_csv("./data/brain.csv").columns) 
#len(brain[brainCol].columns)


/var/folders/k5/cr7j5c4x0hxgb876c543f9qr0000gn/T/ipykernel_62072/1927844948.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  brain = brain.drop(brain.columns[[0, 1, 2, 3]], 1)


In [34]:
Neighbor = ['su_risk_p_1', 
'su_risk_p_2_3',
'su_risk_p_4_5', 
'neighborhood1_2_3_p',
'neighborhood_crime_y']

with open('./data/columns/Neighbor', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(Neighbor, filehandle)

In [35]:
family = ['fes_youth',
'fes_p_ss_fc_pr',
'macv_p_ss_fs',
'macv_p_ss_fo',
'macv_p_ss_isr',
'macv_p_ss_fr',
'macv_p_ss_r',
'demo_prnt_age_v2',
'demo_prnt_marital_v2',
'demo_comb_income_v2',
'demo_fam_exp',
'demo_yrs_1',
'demo_yrs_2',
'parent_rules_q1',
'parent_rules_q4',
'parent_rules_q7']

with open('./data/columns/family', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(family, filehandle)

In [ ]:
parent = [
'asr_scr_perstr_t',
'asr_scr_anxdep_t',
'asr_scr_withdrawn_t',
'asr_scr_somatic_t',
'asr_scr_thought_t',
'asr_scr_attention_t',
'asr_scr_aggressive_t',
'asr_scr_rulebreak_t',
'asr_scr_intrusive_t',
'asr_scr_internal_t',
'asr_scr_external_t',
'asr_scr_totprob_t',
'asr_scr_depress_t',
'asr_scr_anxdisord_t',
'asr_scr_somaticpr_t',
'asr_scr_avoidant_t',
'asr_scr_adhd_t',
'asr_scr_antisocial_t',
'asr_scr_inattention_t',
'asr_scr_hyperactive_t']

with open('./data/columns/parent', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(parent, filehandle)

,rsfmri_cor_ngd_df_scs_cde,rsfmri_cor_ngd_df_scs_pt,rsfmri_cor_ngd_df_scs_hp,rsfmri_cor_ngd_df_scs_ag,rsfmri_cor_ngd_df_scs_aa,rsfmri_cor_ngd_df_scs_bs,rsfmri_cor_ngd_dsa_scs_thp,rsfmri_cor_ngd_dsa_scs_cde,rsfmri_cor_ngd_dsa_scs_pt,rsfmri_cor_ngd_dsa_scs_hp,...,rsfmri_cor_ngd_sa_scs_ag,rsfmri_cor_ngd_sa_scs_aa,rsfmri_cor_ngd_sa_scs_bs,rsfmri_cor_ngd_vta_scs_thp,rsfmri_cor_ngd_vta_scs_cde,rsfmri_cor_ngd_vta_scs_pt,rsfmri_cor_ngd_vta_scs_hp,rsfmri_cor_ngd_vta_scs_ag,rsfmri_cor_ngd_vta_scs_aa,rsfmri_cor_ngd_vta_scs_bs
0,-0.153117,0.027206,0.032820,-0.082469,0.234532,0.145833,-0.018936,-0.026335,-0.045649,-0.061369,...,0.033977,0.176136,0.016017,-0.159200,0.096950,-0.105231,0.093810,-0.146213,-0.065730,0.151937
1,0.040035,0.031941,-0.013471,-0.065428,-0.021419,0.067697,0.124591,0.006709,-0.093654,-0.100387,...,0.000060,0.030458,0.078567,-0.167591,-0.089897,0.047835,-0.040634,0.013578,0.071460,0.015810
2,-0.108044,0.059955,-0.031734,-0.029732,0.032723,0.083478,-0.067818,-0.098145,-0.121598,-0.065169,...,0.038526,0.038154,-0.122156,-0.066243,-0.014754,-0.007929,-0.066657,-0.029313,-0.007276,0.091658
3,-0.052134,0.002824,0.111814,-0.048924,0.043096,0.109763,-0.073380,0.063708,-0.030985,-0.022942,...,0.054630,-0.002562,0.071148,-0.062509,0.020820,-0.048073,0.097190,0.075088,0.065463,0.058266
4,-0.051829,-0.093635,0.188045,-0.107743,0.066660,-0.000633,0.261946,-0.049499,-0.070665,0.017614,...,-0.208286,-0.034073,0.005343,-0.197360,-0.046674,-0.022899,0.026314,0.014951,0.072129,0.116322
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11275,0.004183,-0.080022,-0.007151,-0.042689,0.027287,0.099079,-0.006072,-0.022822,-0.043369,-0.023599,...,-0.009521,0.066135,0.109836,0.096130,0.036480,0.038568,0.248696,0.033755,-0.013448,0.112434
11276,-0.102000,-0.075612,0.232200,-0.114184,0.278147,0.232948,-0.054913,0.020878,0.035084,-0.036387,...,0.130922,0.041752,0.125584,0.126389,-0.009587,-0.116175,0.123111,-0.079003,-0.029040,0.107674
11277,-0.039343,-0.112611,0.133796,-0.026431,0.020775,0.196876,0.059272,0.076762,0.040569,-0.057350,...,-0.051546,0.073093,0.051083,-0.019044,0.026323,-0.086471,0.144194,-0.004044,0.035556,0.186812
11278,0.109353,0.065857,-0.104429,0.078904,0.229680,-0.104814,0.054690,-0.000895,-0.289647,-0.106596,...,-0.146889,0.034003,0.004844,0.057078,-0.118433,-0.011791,0.037884,0.066272,0.322386,0.101986
